# WIP: Query the Dataset in S3 using Amazon Athena and AWS Glue Catalog
This will create an Athena table in the Glue Catalog (Hive Metastore) which allows us to query the dataset files in S3.  We will create a table in the Glue Catalog based on the `Amazon Customer Reviews Dataset` in S3.  We define the columns that map to the data, specify how the data is formatted, and provide the location in Amazon S3 for the file(s).


https://iceberg.apache.org/docs/0.13.1/aws/

In [105]:
!curl -O https://repo1.maven.org/maven2/software/amazon/awssdk/bundle/2.15.40/bundle-2.15.40.jar
!curl -O https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.1_2.12/0.13.1/iceberg-spark-runtime-3.1_2.12-0.13.1.jar
!curl -O https://repo1.maven.org/maven2/software/amazon/awssdk/url-connection-client/2.15.40/url-connection-client-2.15.40.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  245M  100  245M    0     0   265M      0 --:--:-- --:--:-- --:--:--  265M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21.0M  100 21.0M    0     0  39.4M      0 --:--:-- --:--:-- --:--:-- 39.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21027  100 21027    0     0   438k      0 --:--:-- --:--:-- --:--:--  446k


In [109]:
!aws s3 cp iceberg-spark-runtime-3.1_2.12-0.13.1.jar s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/iceberg-spark-runtime-3.1_2.12-0.13.1.jar
!aws s3 cp bundle-2.15.40.jar s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/bundle-2.15.40.jar
!aws s3 cp url-connection-client-2.15.40.jar s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/url-connection-client-2.15.40.jar

upload: ./iceberg-spark-runtime-3.1_2.12-0.13.1.jar to s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/iceberg-spark-runtime-3.1_2.12-0.13.1.jar
upload: ./bundle-2.15.40.jar to s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/bundle-2.15.40.jar
upload: ./url-connection-client-2.15.40.jar to s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/url-connection-client-2.15.40.jar


In [111]:
!aws s3 ls s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/

2023-03-16 06:20:16  257939967 bundle-2.15.40.jar
2023-03-16 06:20:14   22123750 iceberg-spark-runtime-3.1_2.12-0.13.1.jar
2023-03-16 06:20:19      21027 url-connection-client-2.15.40.jar


In [113]:
%stop_session

Stopping session: iceberg-to-sagemaker--7ace71e5-b93a-432f-8547-c6041175951d
Stopped session.


In [120]:
%session_id_prefix iceberg-to-sagemaker
%additional_python_modules seaborn,psutil
%number_of_workers 10
%extra_jars s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/bundle-2.15.40.jar,s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/iceberg-spark-runtime-3.1_2.12-0.13.1.jar,s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/url-connection-client-2.15.40.jar
%extra_py_files s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/bundle-2.15.40.jar,s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/iceberg-spark-runtime-3.1_2.12-0.13.1.jar,s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/url-connection-client-2.15.40.jar
%glue_version 3.0

Setting session ID prefix to iceberg-to-sagemaker
Additional python modules to be included:
seaborn
psutil
Previous number of workers: 10
Setting new number of workers to: 10
Extra jars to be included:
s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/bundle-2.15.40.jar
s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/iceberg-spark-runtime-3.1_2.12-0.13.1.jar
s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/url-connection-client-2.15.40.jar
Extra py files to be included:
s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/bundle-2.15.40.jar
s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/iceberg-spark-runtime-3.1_2.12-0.13.1.jar
s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/url-connection-client-2.15.40.jar
Setting Glue version to: 3.0
s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/bundle-2.15.40.jar,s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/iceberg-spark-runtime-3.1_2.12-0.13.1.jar,s3://sagemaker-us-east-1-371366150581/g

In [121]:
# spark.sql.catalog.demo.catalog-impl=org.apache.iceberg.aws.glue.GlueCatalog
# spark.sql.catalog.demo.warehouse=s3://sagemaker-us-east-1-371366150581
# spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions
# spark.sql.catalog.demo=org.apache.iceberg.spark.SparkCatalog
# spark.sql.catalog.glue_catalog.io-impl=org.apache.iceberg.aws.s3.S3FileIO
# spark.sql.catalog.glue_catalog.lock-impl=org.apache.iceberg.aws.glue.DynamoLockManager
# spark.sql.catalog.glue_catalog.lock.table=demo-iceberg-gis

# sc = spark.sparkContext.getConf()
# sc = sc.set("spark.sql.catalog.demo.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog")
# sc = sc.set("spark.sql.catalog.demo.warehouse", "s3://sagemaker-us-east-1-371366150581")
# sc = sc.set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
# sc = sc.set("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog")
# sc = sc.set("spark.sql.catalog.glue_catalog.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
# sc = sc.set("spark.sql.catalog.glue_catalog.lock-impl", "org.apache.iceberg.aws.glue.DynamoLockManager")
# sc = sc.set("spark.sql.catalog.glue_catalog.lock.table", "demo-iceberg-gis")

In [123]:
%%configure
{
    "conf": "spark.sql.catalog.glue_catalog.catalog-impl=org.apache.iceberg.aws.glue.GlueCatalog --conf spark.sql.catalog.glue_catalog.warehouse=s3://sagemaker-us-east-1-371366150581/iceberg/ --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions --conf spark.sql.catalog.glue_catalog=org.apache.iceberg.spark.SparkCatalog --conf spark.sql.catalog.glue_catalog.io-impl=org.apache.iceberg.aws.s3.S3FileIO --conf spark.sql.catalog.glue_catalog.lock-impl=org.apache.iceberg.aws.glue.DynamoLockManager --conf spark.sql.catalog.glue_catalog.lock.table=demo-iceberg-gis",
}

The following configurations have been updated: {'conf': 'spark.sql.catalog.glue_catalog.catalog-impl=org.apache.iceberg.aws.glue.GlueCatalog --conf spark.sql.catalog.glue_catalog.warehouse=s3://sagemaker-us-east-1-371366150581/iceberg/ --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions --conf spark.sql.catalog.glue_catalog=org.apache.iceberg.spark.SparkCatalog --conf spark.sql.catalog.glue_catalog.io-impl=org.apache.iceberg.aws.s3.S3FileIO --conf spark.sql.catalog.glue_catalog.lock-impl=org.apache.iceberg.aws.glue.DynamoLockManager --conf spark.sql.catalog.glue_catalog.lock.table=demo-iceberg-gis'}


In [1]:
spark

Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::371366150581:role/SageMakerRepoRole
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 10
Session ID: iceberg-to-sagemaker--8c73d653-ddf7-4067-8c2e-5248c9002d36
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
--additional-python-modules seaborn,psutil
--extra-jars s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/bundle-2.15.40.jar,s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/iceberg-spark-runtime-3.1_2.12-0.13.1.jar,s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/url-connection-client-2.15.40.jar
--extra-py-files s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/bundle-2.15.40.jar,s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/iceberg-spark-runtime-3.1_2.12-0.13.1.jar,s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/url-connection-client-2.15.40.jar

In [2]:
spark.sql("CREATE DATABASE IF NOT EXISTS glue_catalog.reviews")

DataFrame[]


In [3]:
df_parquet = spark.read.parquet("s3://amazon-reviews-pds/parquet/product_category=Books/*.parquet")

In [4]:
df_parquet.createOrReplaceTempView("tmp_book_reviews_delta")

In [5]:
query = f"""
CREATE TABLE glue_catalog.reviews.book_reviews_delta
USING iceberg
AS SELECT * FROM tmp_book_reviews_delta
"""
spark.sql(query)

DataFrame[]


You are already connected to a glueetl session 50436d90-cf93-45a4-9e8b-c3b5fc381c4c.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


The following configurations have been updated: {'conf': {'spark.sql.catalog.demo': 'org.apache.iceberg.spark.SparkCatalog', 'spark.sql.catalog.demo.catalog-impl': 'org.apache.iceberg.aws.glue.GlueCatalog', 'spark.sql.catalog.demo.warehouse': 's3://sagemaker-us-east-1-371366150581', 'spark.sql.extensions': 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions'}}


In [2]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

svmem(total=33242578944, available=31339290624, percent=5.7, used=1490272256, free=14115483648, active=802324480, inactive=16294055936, buffers=2768896, cached=17634054144, shared=839680, slab=1531084800)


In [4]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

ModuleNotFoundError: No module named 'sagemaker'


In [4]:
from pyathena import connect
import pandas as pd

In [5]:
%store -r setup_dependencies_passed

In [6]:
try:
    setup_dependencies_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [7]:
print(setup_dependencies_passed)

True


In [8]:
if not setup_dependencies_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [9]:
%store -r role

# Create Athena Table for Parquet Data

#### Dataset columns

- `marketplace`: 2-letter country code (in this case all "US").
- `customer_id`: Random identifier that can be used to aggregate reviews written by a single author.
- `review_id`: A unique ID for the review.
- `product_id`: The Amazon Standard Identification Number (ASIN).  `http://www.amazon.com/dp/<ASIN>` links to the product's detail page.
- `product_parent`: The parent of that ASIN.  Multiple ASINs (color or format variations of the same product) can roll up into a single parent.
- `product_title`: Title description of the product.
- `product_category`: Broad product category that can be used to group reviews (in this case digital videos).
- `star_rating`: The review's rating (1 to 5 stars).
- `helpful_votes`: Number of helpful votes for the review.
- `total_votes`: Number of total votes the review received.
- `vine`: Was the review written as part of the [Vine](https://www.amazon.com/gp/vine/help) program?
- `verified_purchase`: Was the review from a verified purchase?
- `review_headline`: The title of the review itself.
- `review_body`: The text of the review.
- `review_date`: The date the review was written.



In [10]:
s3_staging_dir = "s3://{}/athena/query-cache".format(bucket)

In [11]:
database_name = "default"
table_name = "amazon_reviews_parquet"

In [12]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [13]:
# SQL statement to execute
statement = """
    CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
      marketplace string, 
      customer_id string, 
      review_id string, 
      product_id string, 
      product_parent string, 
      product_title string, 
      star_rating int, 
      helpful_votes int, 
      total_votes int, 
      vine string, 
      verified_purchase string, 
      review_headline string, 
      review_body string, 
      review_date int, 
      year int)
    PARTITIONED BY (product_category string)
    ROW FORMAT SERDE 
      'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
    STORED AS INPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' 
    OUTPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
    LOCATION
      's3://amazon-reviews-pds/parquet/'
""".format(
    database_name, table_name
)

print(statement)

pd.read_sql(statement, conn)


    CREATE EXTERNAL TABLE IF NOT EXISTS default.amazon_reviews_parquet (
      marketplace string, 
      customer_id string, 
      review_id string, 
      product_id string, 
      product_parent string, 
      product_title string, 
      star_rating int, 
      helpful_votes int, 
      total_votes int, 
      vine string, 
      verified_purchase string, 
      review_headline string, 
      review_body string, 
      review_date int, 
      year int)
    PARTITIONED BY (product_category string)
    ROW FORMAT SERDE 
      'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
    STORED AS INPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' 
    OUTPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
    LOCATION
      's3://amazon-reviews-pds/parquet/'



""


# Analyze and Build the Partitions

In [14]:
statement = """
    MSCK REPAIR TABLE {}.{}
""".format(
    database_name, table_name
)

print(statement)

pd.read_sql(statement, conn)


    MSCK REPAIR TABLE default.amazon_reviews_parquet



""


# Verify The Table Has Been Created Succesfully

In [15]:
statement = "SHOW TABLES in {}".format(database_name)

df_tables = pd.read_sql(statement, conn)
df_tables

,tab_name
0,amazon_reviews_parquet


# Run A Sample Query

In [16]:
product_category = "Digital_Software"

statement = """
    SELECT * FROM {}.{}
    WHERE product_category = '{}' LIMIT 100
""".format(
    database_name, table_name, product_category
)

print(statement)


    SELECT * FROM default.amazon_reviews_parquet
    WHERE product_category = 'Digital_Software' LIMIT 100



In [17]:
df = pd.read_sql(statement, conn)
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,year,product_category
0,US,29736021,R3AKKHBV6K3CBH,B00E7XA7KY,189774198,Quickbooks Pro,1,1,1,N,N,Do not buy if you have a MAC and are currently...,for some reason the geniuses over at quickbook...,16171,2014,Digital_Software
1,US,46908092,R2RMBHTWTZLE40,B008SCNLEY,643150354,Norton 360 2013 - 1 User / 3 PC,4,0,0,N,Y,Been using it for years...,"What can I say, Norton can be a bloatware, but...",15900,2013,Digital_Software
2,US,42768397,R3PBHG9KZQUAUW,B00E7X9WZU,915569085,Quicken Premier 2,4,0,0,N,Y,User friendly,"It took a few minutes to set up, after that it...",16171,2014,Digital_Software
3,US,51342536,R10DR245KPSKQ3,B008S0IMCC,534964191,Quicken Deluxe 2013,1,0,0,N,Y,"Wow- Registration does not work, so neither do...",My comment to the Quicken problem report:<br /...,15900,2013,Digital_Software
4,US,3396648,R35TP4JMIUJTYN,B00H9A60O4,608720080,Avast Free Antivirus 2015 [Download],5,0,0,N,N,Avast Review,I have Avast free version and love it . I like...,16171,2014,Digital_Software


In [18]:
if df.empty:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA.")
    print("LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.             ")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [19]:
if "amazon_reviews_parquet" in df_tables.values and not df.empty:
    ingest_create_athena_table_parquet_passed = True

In [20]:
%store ingest_create_athena_table_parquet_passed

Stored 'ingest_create_athena_table_parquet_passed' (bool)


# Review the New Athena Table in the Glue Catalog

In [21]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#table:catalog={};name=amazon_reviews_parquet;namespace=default">AWS Glue Catalog</a></b>'.format(
            region, account_id
        )
    )
)

# Store Variables for the Next Notebooks

In [22]:
%store

Stored variables and their in-db values:
ingest_create_athena_table_parquet_passed             -> True
role                                                  -> 'arn:aws:iam::371366150581:role/SageMakerRepoRole'
s3_private_path_tsv                                   -> 's3://sagemaker-us-east-1-371366150581/amazon-revi
s3_public_path_tsv                                    -> 's3://amazon-reviews-pds/tsv'
setup_dependencies_passed                             -> True


# Release Resources

In [23]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>